<a href="https://colab.research.google.com/github/HeatherDriver/MathGraph/blob/main/03_NER_Case_Tag_Dictionary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install intervaltree

  Preparing metadata (setup.py) ... done
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=d4b13dd4160822f97cc03afe5c579a007e899a8d863bec84d83afaffae438db4
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from google.colab import drive, userdata
import pickle
import random
import re
from collections import defaultdict
from intervaltree import Interval, IntervalTree
from collections import Counter
import json

In [3]:
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files


In [4]:
# Define nested_dict function
def nested_dict():
  return defaultdict(nested_dict)

# Define file read function
def read_pickle(dict_file):
  with open(dict_file, 'rb') as file:
    return pickle.load(file)

# Split CamelCase to s e p a r a t e d text
def split_camel_case(text):
    # Use regex to insert a space before uppercase letters preceded by lowercase letters
    return re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)

In [5]:
# Existing tags from DistilBERT
tag_checkpoint = "dslim/distilbert-NER"
tag_tokenizer = AutoTokenizer.from_pretrained(tag_checkpoint, do_lower_case=False)
tag_model = AutoModelForTokenClassification.from_pretrained(tag_checkpoint)

tags = pipeline("ner", model=tag_model, tokenizer=tag_tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/926 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

Device set to use cpu


In [6]:
tag_tokenizer.is_fast

True

In [7]:
# Methods built from DistilBERT NER

def idx2string(text):
    # Split on whitespace or punctuation, punctuation is used for tokens
    words = re.findall(r'\b\w+\b|[^\s\w]', text)
    my_dict = {i: word for i, word in enumerate(words)}
    return my_dict

def tokenidx2words(ner_results):
  my_dict = dict()
  for sub_dict in ner_results:
    word, index = sub_dict['word'], sub_dict['index']
    my_dict[index] = word
  return my_dict

def tokenidx2entity(ner_results):
  my_dict = dict()
  for sub_dict in ner_results:
    entity, index = sub_dict['entity'], sub_dict['index']
    my_dict[index] = entity
  return my_dict

def idx2wordpos(text, idx2string):
  my_dict = {}
  current_pos = 0
  for idx, word in idx2string.items():
    start = text.index(word, current_pos)
    end = start + len(word)
    my_dict[idx] = (start, end)
    current_pos = end
  return my_dict

def tokenidx2entityword(example2wordidx, ner_results):
  # Create an IntervalTree
  tree = IntervalTree(Interval(start, end, key) for key, (start, end) in example2wordidx.items())

  my_dict = dict()
  for sub_dict in ner_results:
    tag_start, tag_end = sub_dict['start'], sub_dict['end']
    tag_entity = sub_dict['entity']
    token_idx = sub_dict['index']

    overlapping_intervals = tree[tag_start : tag_end] # Intervals which contain the entitry recognised
    for interval in overlapping_intervals:
      my_dict[token_idx] = {'entity' : tag_entity, 'word_idx': interval.data}

  return my_dict

In [36]:
def idx2token_and_id(idx2string_dict, token_list, input_ids_list):
  my_token_dict = dict()
  token_index = 0 # start of tokens list
  # match word to the start of the token
  for key, value in idx2string_dict.items():
    reconstructed_word = ""
    start_token = None
    start_input_id = None

    # Loop through tokens to find the starting token
    while token_index < len(token_list):
      token = token_list[token_index]

      # strip '##' in subwords
      if token.startswith("##"):
        reconstructed_word += token[2:]  # Add subword without "##"
      else:
        if reconstructed_word:  # Reset if a new word starts
          reconstructed_word = ""
        reconstructed_word += token
        start_token = token
        start_input_id = input_ids_list[token_index]
      # Break if reconstructed
      if reconstructed_word == value:
        my_token_dict[key] = (start_token, start_input_id)
        token_index += 1  # Move to the next token
        break
      token_index += 1
  return my_token_dict

In [40]:
# Get naming convention from scraped data
topics = read_pickle("topics.pkl")

print(f"topic_dict dictionary length: {len(topics)}")
random_sample = random.sample(list(topics.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n{value}")

topic_dict dictionary length: 10
Applied Mathematics:
['Business', 'Complex Systems', 'Control Theory', 'Data Visualization', 'Dynamical Systems', 'Engineering', 'Ergodic Theory', 'Game Theory', 'Information Theory', 'Inverse Problems', 'Numerical Methods', 'Optimization', 'Population Dynamics', 'Signal Processing']
Geometry:
['Algebraic Geometry', 'Centroids', 'Combinatorial Geometry', 'Computational Geometry', 'Continuity Principle', 'Coordinate Geometry', 'Curves', 'Differential Geometry', 'Dissection', 'Distance', 'Division Problems', 'Ergodic Theory', 'General Geometry', 'Geometric Construction', 'Geometric Duality', 'Geometric Inequalities', 'Inversive Geometry', 'Line Geometry', 'Multidimensional Geometry', 'Non-Euclidean Geometry', 'Plane Geometry', 'Points', 'Projective Geometry', 'Rigidity', 'Sangaku Problems', 'Solid Geometry', 'Surfaces', 'Symmetry', 'Transformations', 'Trigonometry']
Calculus and Analysis:
['Calculus', 'Calculus of Variations', 'Catastrophe Theory', 'Compl

In [42]:
# Get naming convention from scraped data
sub_topics = read_pickle("sub_topics.pkl")

my_dict = dict()
for key, value in sub_topics.items():
  split_key = split_camel_case(key)
  my_dict[split_key] = value

sub_topics = my_dict

print(f"sub_topics dictionary length: {len(sub_topics)}")

for key, value in random_sample:
  print(f"{key}:\n{sub_topics[key]}")

sub_topics dictionary length: 10
Applied Mathematics:
['Accounting', 'Actuarial Mathematics', 'Economics', 'Finance', 'Cellular Automata', 'Chaos', 'Fractals', 'Control Theory', 'Filter', 'Jordan Canonical Form', 'Kalman Filter', "Little's Law", 'Pontryagin Maximum Principle', 'Riccati Differential Equation', 'Schur Decomposition', 'Similarity Transformation', 'Transfer Function', 'Anti-Aliasing', 'Antialiasing', 'Bar Chart', 'Binary Plot', 'Cobweb Plot', 'Contour Line', 'Contour Plot', 'Data Cube', 'de Finetti Diagram', 'Equipotential Curve', 'Function Graph', 'Inside-Outside Plot', 'Level Curve', 'Level Set', 'Level Surface', 'Moiré Pattern', 'Ordinate', 'Phase Curve', 'Phase Flow', 'Phase Plane', 'Phase Portrait', 'Phase Space', 'Pie Chart', 'Poincaré Section', 'Pólya Plot', 'Recurrence Plot', 'Saunders Graphic', 'Scatter Diagram', 'Scatter Plot', 'Scatterplot', 'Simplex Plot', 'Slope Field', 'Sparkline', "Strang's Strange Figures", 'Surface of Section', 'Ternary Diagram', 'Ternary 

In [43]:
# Merge Topics and Subtopics to one dictionary
for key, value in topics.items():
  if key in sub_topics and sub_topics[key] is not None:
    value.extend(sub_topics[key])
    # Convert to set and back to list to remove duplicates and sort
    new_value = list(set(value))
    new_value.sort()
    topics[key] = new_value

In [44]:
print(f"topics dictionary length: {len(topics)}")

for key, value in random_sample:
  print(f"{key}:\n{topics[key]}")

topics dictionary length: 10
Applied Mathematics:
['2x mod 1 Map', 'Accounting', 'Accumulation Point', 'Action', 'Actuarial Mathematics', 'Adiabatic Invariant', 'All-to-All Communication', 'Allometric', 'Ambrose-Kakutani Theorem', 'Angel Problem', 'Annealing', 'Anonymous', 'Anosov Automorphism', 'Anosov Diffeomorphism', 'Anosov Flow', 'Anosov Map', 'Ant Colony Algorithm', 'Anti-Aliasing', 'Antialiasing', 'Approximation Theory', "Arnold's Cat Map", "Arrow's Impossibility Theorem", "Arrow's Paradox", 'Asymptotic Equipartition Property', 'Auction', 'Average Power', 'Axiom A Flow', "Baker's Map", 'Banzhaf Power Index', 'Bar Chart', 'Bifurcation', 'Bifurcation Theory', 'Biggest Little Hexagon', 'Biggest Little Polygon', 'Binary Plot', 'Biotic Potential', "Birkhoff's Ergodic Theorem", "Birkhoff's Inequality", 'Borel Determinacy Theorem', 'Branch and Bound Algorithm', 'Broadcasting', 'Building Problem', 'Business', 'Byzantine Generals Problem', 'Cake Cutting', 'Carrying Capacity', 'Cascade', 

In [45]:
alg_2 = read_pickle("alg_2.pkl")
alg_3 = read_pickle("alg_3.pkl")

# merge alg_2 and alg_3 into 1 dictionary
alg_dict = dict()

for key, value in alg_2.items():
  alg_dict[key] = value

for key, value in alg_3.items():
  alg_dict[key] = value

print(f"alg_dict dictionary length: {len(alg_dict)}")
random_sample = random.sample(list(alg_dict.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n{value}")

alg_dict dictionary length: 18
Determinants:
['Casoratian', "Cauchy's Determinant Theorem", 'Cayley-Menger Determinant', 'Chió Pivotal Condensation', 'Circulant Determinant', 'Cofactor', 'Condensation', 'Determinant', 'Determinant Expansion by Minors', 'Determinant Identities', 'Determinant Theorem', 'Gram Determinant', "Gram's Inequality", "Hadamard's Maximum Determinant Problem", "Hadamard's Theorem", 'Hafner-Sarnak-McCurley Constant', 'Hessian', 'Hill Determinant', 'Hyperdeterminant', 'Inversion Number', "Jacobi's Determinant Identity", "Jacobi's Theorem", 'Jacobian', 'Mills-Robbins-Rumsey Determinant Formula', 'Minor', 'PfaffianPivotal Condensation', "Schweins's Theorem", 'Stäckel Determinant', "Sylvester's Determinant Identity", 'Vandermonde Determinant']
Lie Theory:
['Lie Algebra', 'Lie Groups']
Matrix Types:
['Array', 'Column Space', 'Commuting Matrices', 'Congruent Matrices', 'Conjugate Transpose', 'Adjacency Matrix', 'DiagonalMatrixQ', 'Adjoint Matrix', 'Adjugate Matrix', 'Alt

In [46]:
all_topics = dict()
for key, value in alg_dict.items():
  all_topics[key] = value

for key, value in topics.items():
  all_topics[key] = value

all_topics['Linear Algebra'] = all_topics['General Linear Algebra']
del all_topics['General Linear Algebra']

print(f"all_topics dictionary length: {len(all_topics)}")
random_sample = random.sample(list(all_topics.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n{value}")

all_topics dictionary length: 28
Foundations of Mathematics:
['0-Connected', '1-Connected', 'A New Kind of Science', 'AND', 'Abelian Category', 'Absorption Identities', 'Absorption Identity', 'Absorption Law', 'Ackermann Function', 'Additive Category', 'Additive Cellular Automaton', 'Additive Functor', 'Affine Hull', 'Algebra of Random Variables', 'Algebraic Geometry Stack', 'Allegory', 'Almost Everywhere Convergence', 'Apollonian Gasket', 'Arc Component', 'Arcwise-Connected', 'Associative Algebra', 'Axiom', 'Axiom Schema', 'Axiom of Choice', 'Axiom of Extensionality', 'Axiom of Foundation', 'Axiom of Infinity', 'Axiom of Replacement', 'Axiom of Subsets', 'Axiom of the Empty Set', 'Axiom of the Power Set', 'Axiom of the Sum Set', 'Axiom of the Unordered Pair', 'Axiomatic Set Theory', 'Axiomatic System', 'Axioms', 'Axioms of Subsets', 'Basepoint', 'Basis Element', 'Bicollared', 'Binary', 'Boolean Function', 'Bootstrap Percolation', 'Borel Hierarchy', 'Borel Measure', 'Borel Set', 'Borel

In [47]:
# Check 1: 1 mapping of keys values
my_dict = dict()
for key, value_list in all_topics.items():
  for value in value_list:
    if value not in my_dict.keys():
      my_dict[value] = [key]
    else:
      my_dict[value].extend([key])

map_to = dict()
for key, value_list in my_dict.items():
  if len(value_list) == 2:
    if 'Recreational Mathematics' in value_list:
      value_list.remove('Recreational Mathematics')
      map_to[key] = value_list[0]

# Create a list of keys to delete to avoid changing dictionary size during iteration
keys_to_delete = [key for key in my_dict if key in map_to]

# Delete the keys
for key in keys_to_delete:
  del my_dict[key]
  # Delete from all_topics
  all_topics['Recreational Mathematics'].remove(key)

In [48]:
# Choose linear algebra topic classification over other types
all_topics_keys_to_remove = dict()
keys_to_delete = list()

for key, value_list in my_dict.items():
  if len(value_list) == 2:
    if 'Determinants' in value_list:
      value_list.remove('Determinants')
      all_topics_keys_to_remove[value_list[0]] = key
      keys_to_delete.append(key)

# Delete the keys
for key in keys_to_delete:
  del my_dict[key]

for key, value in all_topics_keys_to_remove.items():
  # Delete from all_topics
  all_topics[key].remove(value)

In [49]:
my_dict = dict()
for key, value_list in all_topics.items():
  for value in value_list:
    if value not in my_dict.keys():
      my_dict[value] = [key]
    else:
      my_dict[value].extend([key])

In [50]:
to_remove = {
    "Fundamental Matrix Subspaces": ['Matrix Decomposition', 'Matrix Operations'],
    "Matrix Diagonalization": ['Matrix Decomposition', 'Matrix Eigenvalues'],
    "Stochastic Matrix": ['Algebra', 'Probability and Statistics'],
    "Hadamard Matrix": ['Algebra', 'Foundations of Mathematics', 'Recreational Mathematics', 'Discrete Mathematics'],
    "Polynomial Matrix": [ 'Algebra', 'Calculus and Analysis'],
    "Sylvester Matrix": ['Algebra', 'Calculus and Analysis'],
    "Matrix Polynomial": ['Algebra', 'Calculus and Analysis']
    }

for key, value_list in to_remove.items():
  for val in value_list:
    all_topics[val].remove(key)

my_dict = dict()
for key, value_list in all_topics.items():
  for value in value_list:
    if value not in my_dict.keys():
      my_dict[value] = [key]
    else:
      my_dict[value].extend([key])

In [51]:
to_remove = {
    "Lie Algebra": ['Lie Theory', 'Lie Algebra']
    ,"Fredholm's Theorem": ['Linear Independence', 'Matrix Properties']
    ,"Jacobi Identities": ['Lie Algebra', 'Lie Groups']
    ,"Flag Manifold": ['Lie Groups', 'Calculus and Analysis']
    ,"Lie-Type Group": ['Lie Groups', 'Discrete Mathematics']
    ,"Special Linear Group": ['Lie Groups', 'Discrete Mathematics']
    ,"Fundamental Theorem of Linear Algebra": ['Matrix Decomposition', 'Matrix Operations']
    ,"Schur Decomposition": ['Matrix Eigenvalues', 'Applied Mathematics']
    ,"Conjugate Transpose": ['Matrix Types', 'Calculus and Analysis']
    ,"Analytic Torsion": ['Calculus and Analysis', 'Topology']
    ,"Byzantine Generals Problem": ['Algebra', 'Applied Mathematics']
    ,"Code": ['Algebra', 'Foundations of Mathematics']
    ,"Commutative Diagram": ['Algebra', 'Foundations of Mathematics']
    ,"Contractible": ['Foundations of Mathematics', 'Topology']
    ,"Cyclotomic Polynomial": ['Foundations of Mathematics', 'Discrete Mathematics']
    ,"Elliptic Curves": ['Algebra', 'Geometry']
    ,"Encoding": ['Algebra', 'Foundations of Mathematics']
    ,"Endomorphism": ['Algebra', 'Foundations of Mathematics', 'Applied Mathematics', 'Geometry']
    ,"Ergodic Theory": ['Algebra', 'Applied Mathematics', 'Geometry', 'Calculus and Analysis']
    ,"Gerbe": [ 'Foundations of Mathematics', 'Topology']
    ,"Gröbner Basis": ['Algebra', 'Discrete Mathematics']
    ,"Icosahedral Equation": ['Algebra', 'Calculus and Analysis']
    ,"Kac Formula": ['Algebra', 'Probability and Statistics']
    ,"Minimax Polynomial": ['Probability and Statistics', 'Calculus and Analysis']
    ,"Modularity Theorem": ['Algebra', 'Number Theory']
    ,"Noise": ['Algebra', 'Applied Mathematics']
    ,"Normal Polynomial": ['Calculus and Analysis', 'Number Theory']
    ,"Octahedral Equation": ['Geometry', 'Calculus and Analysis']
    ,"Polynomial Sequence": ['Algebra', 'Calculus and Analysis']
    ,"Power Polynomial": ['Calculus and Analysis', 'Discrete Mathematics']
    ,"Real Vector Space": ['Algebra', 'Topology']
    ,"Reidemeister Torsion": ['Calculus and Analysis', 'Topology']
    ,"Taniyama-Shimura Conjecture": ['Algebra', 'Topology']
    ,"Tetrahedral Equation": ['Algebra', 'Geometry']
    ,"Vector Field": ['Applied Mathematics', 'Topology']
    ,"Whitehead Torsion": ['Algebra', 'Calculus and Analysis']
    ,"Zero Map": ['Algebra', 'Foundations of Mathematics', 'Calculus and Analysis']
    ,"Borel Hierarchy": ['Foundations of Mathematics', 'Topology']
    ,"Busy Beaver": ['Foundations of Mathematics', 'Recreational Mathematics']
    ,"Circle Packing": ['Foundations of Mathematics', 'Recreational Mathematics', 'Discrete Mathematics']
    ,"Closed Map": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Collatz Problem": ['Foundations of Mathematics', 'Recreational Mathematics']
    ,"Complete Metric Space": ['Foundations of Mathematics', 'Topology']
    ,"Contingent Cone": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Dense": ['Foundations of Mathematics', 'Topology']
    ,"Domain": ['Foundations of Mathematics', 'Calculus and Analysis', 'Topology']
    ,"Endpoint": ['Foundations of Mathematics', 'Geometry']
    ,"F_sigma Set": ['Foundations of Mathematics', 'Topology']
    ,"First Category": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"G_delta Set": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Generalized Reeb Component": ['Foundations of Mathematics', 'Topology']
    ,"Helly's Theorem": ['Foundations of Mathematics', 'Geometry']
    ,"Hofstadter's Q-Sequence": ['Foundations of Mathematics', 'Discrete Mathematics']
    ,"Hofstadter-Conway $10,000 Sequence": ['Foundations of Mathematics', 'Discrete Mathematics']
    ,"Injection": ['Foundations of Mathematics', 'Topology']
    ,"Lebesgue Covering Dimension": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Logistic Map": ['Foundations of Mathematics', 'Applied Mathematics', 'Discrete Mathematics']
    ,"Logistic Map--r=2": ['Foundations of Mathematics', 'Applied Mathematics']
    ,"Logistic Map--r=4": ['Foundations of Mathematics', 'Applied Mathematics']
    ,"Metric Topology": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Metrizable Topology": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Net": ['Foundations of Mathematics', 'Probability and Statistics']
    ,"Nice Vector Field": ['Foundations of Mathematics', 'Topology']
    ,"Pseudoconcave Function": ['Foundations of Mathematics', 'Topology']
    ,"Pseudoconvex Function": ['Foundations of Mathematics', 'Topology']
    ,"Quasi-Concave Function": ['Foundations of Mathematics', 'Topology']
    ,"Quasi-Convex Function": ['Foundations of Mathematics', 'Topology']
    ,"Reeb Component": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Reebless": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Stochastic": ['Foundations of Mathematics', 'Applied Mathematics', 'Geometry', 'Number Theory']
    ,"Surjection": ['Foundations of Mathematics', 'Calculus and Analysis']
    ,"Sutured Manifold": ['Foundations of Mathematics', 'Topology']
    ,"Taut Foliation": ['Foundations of Mathematics', 'Topology']
    ,"Transversely Orientable Foliation": ['Foundations of Mathematics', 'Topology']
    ,"Uniform Convexity": ['Foundations of Mathematics', 'Topology']
    ,"Zero Set": ['Foundations of Mathematics', 'Topology']
    ,"Lévy Flight": ['Probability and Statistics', 'Applied Mathematics', 'Geometry']
    ,"Lévy Walk": ['Probability and Statistics', 'Applied Mathematics', 'Geometry']
    ,"Quantum Stochastic Calculus": ['Probability and Statistics', 'Geometry', 'Number Theory']
    ,"Sampling": ['Applied Mathematics', 'Discrete Mathematics']
    ,"Seed": ['Probability and Statistics', 'Calculus and Analysis']
    ,"2x mod 1 Map": ['Calculus and Analysis', 'Discrete Mathematics']
    ,"Anosov Flow": ['Calculus and Analysis', 'Topology']
    ,"Axiom A Flow": ['Calculus and Analysis', 'Topology']
    ,"Cake Cutting": ['Geometry', 'Discrete Mathematics']
    ,"Dye's Theorem": ['Applied Mathematics', 'Geometry']
    ,"Ergodic Measure": ['Applied Mathematics', 'Geometry']
    ,"Geodesic Flow": ['Calculus and Analysis', 'Topology']
    ,"KMS Condition": ['Applied Mathematics', 'Geometry', 'Calculus and Analysis']
    ,"Kronecker-Weyl Theorem": ['Applied Mathematics', 'Geometry']
    ,"Kubo-Martin-Schwinger Condition": ['Geometry', 'Calculus and Analysis', 'Number Theory']
    ,"Lévy Process": ['Geometry', 'Number Theory']
    ,"Number Partitioning Problem": ['Applied Mathematics', 'Geometry']
    ,"Ornstein's Theorem": ['Geometry', 'Number Theory']
    ,"Pesin Theory": ['Geometry', 'Number Theory']
    ,"Phase Space": ['Applied Mathematics', 'Topology']
    ,"Poisson Process": ['Geometry', 'Number Theory']
    ,"Smale Horseshoe Map": ['Applied Mathematics', 'Calculus and Analysis']
    ,"Connex": ['Geometry', 'Calculus and Analysis']
    ,"Dehn Invariant": ['Geometry', 'Recreational Mathematics']
    ,"Integer Triangle": ['Geometry', 'Recreational Mathematics']
    ,"Kobon Triangle": ['Geometry', 'Recreational Mathematics']
    ,"Pizza Theorem": ['Geometry', 'Recreational Mathematics']
    ,"Subset Sum Problem": ['Geometry', 'Discrete Mathematics']
    ,"Banach Space": ['Recreational Mathematics', 'Topology']
    ,"Hilbert Space": ['Recreational Mathematics', 'Topology']
    ,"Perrin Sequence": ['Recreational Mathematics', 'Discrete Mathematics']
    ,"Pi": ['Recreational Mathematics', 'Discrete Mathematics']
}

for key, value_list in to_remove.items():
  for val in value_list:
    all_topics[val].remove(key)

my_dict = dict()
for key, value_list in all_topics.items():
  for value in value_list:
    if value not in my_dict.keys():
      my_dict[value] = [key]
    else:
      my_dict[value].extend([key])

# Final 1:1 check
my_dict = dict()
for key, value_list in my_dict.items():
  if len(value_list) != 1:
    print(f"{key}: {value_list}")

In [52]:
def capitalise_key(key):
  words = key.split()
  if len(words) == 1:
    to_return = words[0].capitalize()
  else:
    first = words[0].capitalize()
    rest = words[1:]
    to_return = ' '.join(rest).lower()
    to_return = f"{first} {to_return}"
  return to_return

In [53]:
def creates_bioes_labels(ner_mapper_list, key):
  # Generate BIOES-style labels for a list of values, corresponding to the input.
  n = len(ner_mapper_list)
  if n == 1:
    labels_list = [f'S-{key}']
  else:
    labels_list = [f'B-{key}']  # Labels start with 'B-ENTITY_NAME'
    labels_list.extend([f'I-{key}'] * (n - 2))  # middle tokens are 'I-ENTITY_NAME'
    labels_list.append(f'E-{key}')  # End with 'E-ENTITY_NAME'
  return labels_list

In [63]:
def generate_ner_tags(classification_dict):
  # Generates hierarchical tags based on the key
  transformations = {
        "lower": str.lower,
        "upper": str.upper,
        "title": str.title,
        "capitalise": capitalise_key,
  }
  my_dict = dict()
  for key, value_list in classification_dict.items():
    key = key.replace(" ", "_").upper()
    for value in value_list:
      transformed_values = {}
      for name, transform_fn in transformations.items():
        transformed_values[name] = transform_fn(value)

      for value in transformed_values.values():
        value = value.replace("\n", " ") #Removing new line in key
        value = value.replace("--", "-") #Removing double-dashes

        text_index = idx2string(value)
        token_list = tag_tokenizer(value).tokens()
        input_id_list = tag_tokenizer.encode(value)
        token_id_index = idx2token_and_id(text_index, token_list, input_id_list)
        my_dict[value] = {
          "text_index": text_index,
          "tokens" : token_list,
          "input_ids" : input_id_list,
          }
        ner_mapper = [tup[1] for key, tup in token_id_index.items()]
        for i, (k, v) in enumerate(my_dict[value]['text_index'].items()):
          my_dict[value]['ner_mapper'] = ner_mapper
          my_dict[value]['ner_tags'] = creates_bioes_labels(ner_mapper, key)
  return my_dict

In [64]:
# Generate the NER tags per dictionary
_all_classifications = generate_ner_tags(all_topics)

In [65]:
print(f"topic_dict_ner dictionary length: {len(_all_classifications)}")
random_sample = random.sample(list(_all_classifications.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n{value}")

topic_dict_ner dictionary length: 21696
disk:
{'text_index': {0: 'disk'}, 'tokens': ['[CLS]', 'disk', '[SEP]'], 'input_ids': [101, 10437, 102], 'ner_mapper': [10437], 'ner_tags': ['S-TOPOLOGY']}
quadratic curve:
{'text_index': {0: 'quadratic', 1: 'curve'}, 'tokens': ['[CLS]', 'q', '##uad', '##ratic', 'curve', '[SEP]'], 'input_ids': [101, 186, 18413, 21961, 7660, 102], 'ner_mapper': [186, 7660], 'ner_tags': ['B-ALGEBRA', 'E-ALGEBRA']}
Elliptic space:
{'text_index': {0: 'Elliptic', 1: 'space'}, 'tokens': ['[CLS]', 'El', '##liptic', 'space', '[SEP]'], 'input_ids': [101, 2896, 24021, 2000, 102], 'ner_mapper': [2896, 2000], 'ner_tags': ['B-GEOMETRY', 'E-GEOMETRY']}


In [67]:
all_classifications = dict()

for key, sub_dict in _all_classifications.items():
  ner = list('O' * len(sub_dict['input_ids']))

  for match_input_id, ner_tag in zip(sub_dict['ner_mapper'], sub_dict['ner_tags']):
    idx = sub_dict['input_ids'].index(match_input_id)
    ner[idx] = ner_tag
    ner = [i.replace('_', '-') for i in ner]
    all_classifications[key] = {
        'text_index': sub_dict['text_index'],
        'tokens': sub_dict['tokens'],
        'input_ids': sub_dict['input_ids'],
        'ner_mapper': sub_dict['ner_mapper'],
        'ner_tags': ner,
    }

In [68]:
print(f"topic_dict_ner dictionary length: {len(all_classifications)}")

for key, value in random_sample:
  print(f"{key}:\n{all_classifications[key]}")

topic_dict_ner dictionary length: 21633
disk:
{'text_index': {0: 'disk'}, 'tokens': ['[CLS]', 'disk', '[SEP]'], 'input_ids': [101, 10437, 102], 'ner_mapper': [10437], 'ner_tags': ['O', 'S-TOPOLOGY', 'O']}
quadratic curve:
{'text_index': {0: 'quadratic', 1: 'curve'}, 'tokens': ['[CLS]', 'q', '##uad', '##ratic', 'curve', '[SEP]'], 'input_ids': [101, 186, 18413, 21961, 7660, 102], 'ner_mapper': [186, 7660], 'ner_tags': ['O', 'B-ALGEBRA', 'O', 'O', 'E-ALGEBRA', 'O']}
Elliptic space:
{'text_index': {0: 'Elliptic', 1: 'space'}, 'tokens': ['[CLS]', 'El', '##liptic', 'space', '[SEP]'], 'input_ids': [101, 2896, 24021, 2000, 102], 'ner_mapper': [2896, 2000], 'ner_tags': ['O', 'B-GEOMETRY', 'O', 'E-GEOMETRY', 'O']}


In [70]:
file_name = 'all_classifications.pkl'

with open(file_name, 'wb') as file:
  pickle.dump(all_classifications, file)